# * Copyright (C) 2019-20 Advenio Tecnosys Pvt. Ltd.: (All Rights Reserved).

*                                                        NOTICE: 

* All information contained herein is, and remains the property of Advenio Tecnosys and its suppliers,  if any.  The intellectual and technical concepts contained

* herein are proprietary to Advenio Tecnosys and its suppliers and may be covered by Indian, U.S. and Foreign Patents, patents in process, and are protected by trade secret or copyright law.

* Appropriation, redistribution, dissemination of this information or reproduction of this material is strictly forbidden unless prior written permission is obtained from Advenio Tecnosys Pvt. Ltd.

*

* Author: Aman Kumar

*

In [4]:
import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import glob 

In [5]:
# please give you path name to enhance image example is given make sure you follow same path

your_path = "/home/harsh/amk/Image_Quality_Dataset/Test/Good Quality"

In [22]:
saved_path = your_path + " after clahe"
made_folder = saved_path[-len(saved_path.split("/")[-1]):]

if made_folder in os.listdir(saved_path[:-len(saved_path.split("/")[-1])]):
    deleting_folder_path = saved_path[:-len(saved_path.split("/")[-1])]+made_folder+"/"
    for del_img_path in glob.glob(deleting_folder_path+'*'):
        os.remove(del_img_path)
    saved_path = saved_path+'/'    
else:
    os.mkdir(saved_path)
    saved_path = saved_path+'/'

print("Your enhanced images saving path will be ", saved_path)    

Your enhanced images saving path will be  /home/harsh/amk/Image_Quality_Dataset/Test/Good Quality after clahe/


In [23]:
kernel_size_list = [26, 29]

def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

inp = input("Do you want to get original and enhanced image together press Y/y for yes and N/n for No ")

ori_and_clahe = False

if inp =="Y" or inp == "y":
    ori_and_clahe = True
else:
    ori_and_clahe = False

inp = input("Do you want to get kerenl and clip limt press Y/y for yes and N/n for No ")

kernel_and_optim_cl = False

if inp =="Y" or inp == "y":
    kernel_and_optim_cl = True
else:
    kernel_and_optim_cl = False
    

for i, img_path in enumerate(tqdm(glob.glob(your_path+"/*")[50:900:13])):
    optim_cl_wrt_kernel = []
    for kernel_size in kernel_size_list:

        clip_limit = [np.round(i/100, 1) for i in range(10,310,10)]
        psnr_value_wrt_cl = []

        org_img = cv2.imread(img_path)

        if org_img.any() == None:
            continue

        for j, cl in enumerate((clip_limit)):

            #Resize image
            org_img = cv2.resize(org_img, (512,512))

            lab = cv2.cvtColor(org_img, cv2.COLOR_BGR2LAB)

            lab_planes = cv2.split(lab)

            img_before_clahe = lab_planes[0]

            clahe = cv2.createCLAHE(clipLimit=cl,tileGridSize=(kernel_size, kernel_size))

            img_after_clahe = clahe.apply(lab_planes[0])

            psnr_value_wrt_cl.append(psnr(img_before_clahe, img_after_clahe))
        
        optim_cl = clip_limit[::-1][psnr_value_wrt_cl[::-1].index(max(psnr_value_wrt_cl[::-1]))]
        optim_cl_wrt_kernel.append(optim_cl)
        
        if kernel_size == kernel_size_list[-1]:
            
            optim_cl = max(optim_cl_wrt_kernel[::-1])
            kernel_size = kernel_size_list[optim_cl_wrt_kernel.index(optim_cl)]
            
            if kernel_size == 26 and optim_cl > 1.5:
                optim_cl = optim_cl + 0.1

            clahe = cv2.createCLAHE(clipLimit=optim_cl,tileGridSize=(kernel_size,kernel_size))

            lab_planes[0] = clahe.apply(lab_planes[0])

            lab = cv2.merge(lab_planes)

            enhance_img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

            img_name = img_path.split("/")[-1]
            len_of_img_format = len(img_name.split('.')[-1])+1
            len_of_img_format
            
            if kernel_and_optim_cl == True:
                kernel_cl = "[" + str(kernel_size)+ ','+ str(optim_cl)+ "]"
            else:
                kernel_cl = ''

            if  ori_and_clahe == True:

                concat_img = np.concatenate((org_img, enhance_img), axis=1)
                cv2.imwrite(saved_path+img_name[:-len_of_img_format]+kernel_cl+img_name[-len_of_img_format:], concat_img)
            else:
                cv2.imwrite(saved_path+img_name[:-len_of_img_format]+kernel_cl+img_name[-len_of_img_format:], enhance_img)

print("Your enhanced path location is ", saved_path)

Do you want to get original and enhanced image together press Y/y for yes and N/n for No y
Do you want to get kerenl and clip limt press Y/y for yes and N/n for No y


100%|██████████| 66/66 [00:16<00:00,  4.08it/s]

Your enhanced path location is  /home/harsh/amk/Image_Quality_Dataset/Test/Good Quality after clahe/
